In [ ]:
import pandas as pd

df_public_dataset = pd.read_csv('../../data/public_dataset/image_data.csv')
df_public_dataset.head()

In [ ]:
# drop columns image_id	link	author	resolution	date_accessed
df_public_dataset.drop(columns=['image_id', 'link', 'author', 'resolution', 'date_accessed'], inplace=True)
df_public_dataset.head()

In [ ]:
# insert a row at the beginning of the dataframe: 1	https://stock.adobe.com/images/diesel-truck-maz-in-work-closeup-fuel-tank-truck-with-adr-sign-30-1202-on-suburban-asphalted-road-inflammable-liquid-goods-transportation/539794111	Ilya	1000x750	2025-02-01 16:22:57 Ewaldo Nieuwenhuis
# row = {'Image_id': 1, 
#        'Link': 'https://stock.adobe.com/images/diesel-truck-maz-in-work-closeup-fuel-tank-truck-with-adr-sign-30-1202-on-suburban-asphalted-road-inflammable-liquid-goods-transportation/539794111', 
#        'Website': 'Adobe Stock',
#        'Author': 'Ilya', 
#        'Resolution': '1000x750', 
#        'Date_accessed': '2025-02-01 16:22:57', 
#        'Added_by': 'Ewaldo Nieuwenhuis'}
# print(len(df_public_dataset))
# df_public_dataset = pd.concat([pd.DataFrame(row, index=[0]), df_public_dataset], ignore_index=True)
# print(len(df_public_dataset))
# df_public_dataset.head()


# this was a mistake because the first row is equal to the third row so do not add this row, it will start with ID 2 but whatever

In [ ]:
df_public_dataset.iloc[2]['Link']


In [ ]:
# preview some images from a folder

import os
import cv2
import matplotlib.pyplot as plt

path = 'data/public_dataset/images_master_thesis'
images = os.listdir(path)
images = [os.path.join(path, image) for image in images]

for image in images[:5]:
    img = cv2.imread(image)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img)
    plt.show()

In [ ]:
# we need to make annotations for these images so we need to prepare some data for that

# image id = the latest image id from df_public_dataset + 1
# image path = path to dataset on roboflow: https://universe.roboflow.com/masters-thesis-yolo-model/adr-tankers-detection
# we can get the resolution from the image itself
# date accessed = datetime.now()
# added by = Ewaldo Nieuwenhuis
# author = masters-thesis-yolo-model on Roboflow, what a hero

# get the latest image id
latest_image_id = df_public_dataset['Image_id'].max()

# loop over the images and add them to a new dataframe
new_images = []
#get the current datetime
import datetime
for image in images:
    # get the resolution
    img = cv2.imread(image)
    resolution = f'{img.shape[1]}x{img.shape[0]}' # width x height
    datenow = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    new_id = int(latest_image_id + 1)

    row = {'Image_id': new_id, 
           'Link': 'https://universe.roboflow.com/masters-thesis-yolo-model/adr-tankers-detection', 
           'Website': 'Roboflow',
            'Author': 'masters-thesis-yolo-model',
            'Resolution': resolution,
            'Date_accessed': datenow,
            'Added_by': 'Ewaldo Nieuwenhuis'}
    new_images.append(row)
    #store the image in a new folder with 'idx.png' but first check if the file already exists, if it does throw an error
    new_image_path = f'data/public_dataset/images/{new_id}.png'
    if os.path.exists(new_image_path):
        raise ValueError(f'File {new_image_path} already exists')
    cv2.imwrite(new_image_path, img)
    latest_image_id += 1

# add the new images to the dataframe
df_new_images = pd.DataFrame(new_images)
df_new_images.head()


In [ ]:
# concat the new images to the public dataset
df_public_dataset_new = pd.concat([ df_public_dataset, df_new_images], ignore_index=True)
df_public_dataset_new.head()

In [ ]:
latest_image_id = df_public_dataset['Image_id'].max()

new_id = int(latest_image_id + 1)
print(new_id)
new_image_path = f'data/public_dataset/images/{new_id}.png'
print(new_image_path)

In [ ]:
# export the new dataset to a csv file
df_public_dataset_new.to_csv('data/public_dataset/public_dataset.csv', index=False)